In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('extended_stoma_dataset.csv')

In [4]:
data.head()

,Unnamed: 0,dates,id,not_registered,num_caries,num_filling_caries,num_filling_no_caries,num_healthy,num_removed_caused_by_caries,num_removed_other_reasons,...,sealed fissure,uncut tooth,age,nationality,date_of_birth,place_of_birth,previous_place_of_living,bite_type,fluorosis,sum_hygiene_index
0,0,2011-2012,1302.xlsx,4,3,0,0,21,0,0,...,0,0,18.0,русский,11.04.01,Мурманск,Мурманск,ортогнатический,0,2.0
1,1,2012-2013,1302.xlsx,4,3,0,0,21,0,0,...,0,0,18.0,русский,11.04.01,Мурманск,Мурманск,ортогнатический,0,2.0
2,2,2013-2014,1302.xlsx,0,4,0,1,23,0,0,...,0,0,18.0,русский,11.04.01,Мурманск,Мурманск,ортогнатический,0,2.0
3,3,2014-2015,1302.xlsx,0,4,0,3,21,0,0,...,0,0,18.0,русский,11.04.01,Мурманск,Мурманск,ортогнатический,0,2.0
4,4,2015-2016,1302.xlsx,0,2,1,4,22,0,0,...,0,0,18.0,русский,11.04.01,Мурманск,Мурманск,ортогнатический,0,2.0


In [5]:
data['end_year'] = data.dates.apply(lambda a: int(a.split('-')[1]))
data['birth_year'] = data['date_of_birth'].apply(lambda a: int('20'+a.split('.')[-1] ))
data['right_age'] = data[['end_year', 'birth_year']].apply(lambda a: a[0]-a[1], axis=1)
data['kpu'] = data[['num_caries', 'num_filling_caries',
                    'num_filling_no_caries', 
                    'num_removed_caused_by_caries', 'num_removed_other_reasons']].sum(axis=1)


In [75]:
train = data.drop(data[data['dates'] == '2017-2018'].index, axis=0) 
test = data[data['dates'] == '2017-2018']

In [61]:
print(train.shape, test.shape)

(1146, 24) (191, 24)


In [93]:
NUM_FEATURES = ['not_registered', 'num_caries',
       'num_filling_caries', 'num_filling_no_caries', 'num_healthy',
       'num_removed_caused_by_caries', 'num_removed_other_reasons',
       'prosthesis crown veneer', 'sealed fissure', 'uncut tooth', 'fluorosis', 
               'sum_hygiene_index', 'right_age', ]

TARGET = ['kpu']
CAT_FEATURES = ['nationality', 'place_of_birth',
               'previous_place_of_living', 'bite_type', ]


In [77]:
train.columns

Index(['Unnamed: 0', 'dates', 'id', 'not_registered', 'num_caries',
       'num_filling_caries', 'num_filling_no_caries', 'num_healthy',
       'num_removed_caused_by_caries', 'num_removed_other_reasons',
       'prosthesis crown veneer', 'sealed fissure', 'uncut tooth', 'age',
       'nationality', 'date_of_birth', 'place_of_birth',
       'previous_place_of_living', 'bite_type', 'fluorosis',
       'sum_hygiene_index', 'end_year', 'birth_year', 'right_age', 'kpu'],
      dtype='object')

In [109]:

x_mean = train[NUM_FEATURES+['id']].groupby('id').mean()
x_cat = train[CAT_FEATURES+['id']].groupby('id').agg(lambda x:x.value_counts().index[0])
last_names = [c+' last' for c in NUM_FEATURES]
x_last = pd.DataFrame(columns=[c+' last' for c in NUM_FEATURES])
x_last[last_names] = train[NUM_FEATURES+['id']].groupby('id').last()
x_past_target = train[TARGET+['id']].groupby('id').agg(lambda x: x.iloc[:len(x)-1].iloc[-1])
x_past_target.columns = ['kpu_last']
x_past_target['kpu'] = train[TARGET+['id']].groupby('id').agg(lambda x: x.iloc[-1])

In [112]:

x_mean_test = data[NUM_FEATURES+['id']].groupby('id').mean()
x_cat_test = data[CAT_FEATURES+['id']].groupby('id').agg(lambda x:x.value_counts().index[0])
last_names = [c+' last' for c in NUM_FEATURES]
x_last_test = pd.DataFrame(columns=[c+' last' for c in NUM_FEATURES])
x_last_test[last_names] = data[NUM_FEATURES+['id']].groupby('id').last()
x_past_target_test = data[TARGET+['id']].groupby('id').agg(lambda x: x.iloc[:len(x)-1].iloc[-1])
x_past_target_test.columns = ['kpu_last']
x_past_target_test['kpu'] = data[TARGET+['id']].groupby('id').agg(lambda x: x.iloc[-1])

In [110]:
x_past_target

,kpu_last,kpu
id,,
1101.xlsx,7,7
1102.xlsx,4,6
1103.xlsx,5,6
1104.xlsx,8,9
1105.xlsx,1,2
...,...,...
7412.xlsx,3,6
7413.xlsx,3,3
7414.xlsx,0,2


In [80]:
print(x_mean.shape, x_cat.shape, x_last.shape)

(191, 14) (191, 4) (191, 14)


In [111]:
x_train.columns

Index(['not_registered', 'num_caries', 'num_filling_caries',
       'num_filling_no_caries', 'num_healthy', 'num_removed_caused_by_caries',
       'num_removed_other_reasons', 'prosthesis crown veneer',
       'sealed fissure', 'uncut tooth', 'fluorosis', 'sum_hygiene_index',
       'right_age', 'nationality', 'place_of_birth',
       'previous_place_of_living', 'bite_type', 'not_registered last',
       'num_caries last', 'num_filling_caries last',
       'num_filling_no_caries last', 'num_healthy last',
       'num_removed_caused_by_caries last', 'num_removed_other_reasons last',
       'prosthesis crown veneer last', 'sealed fissure last',
       'uncut tooth last', 'fluorosis last', 'sum_hygiene_index last',
       'right_age last'],
      dtype='object')

In [113]:
x_train = pd.concat([x_mean, x_cat, x_last, x_past_target['kpu_last']], axis=1)
y_train = x_past_target['kpu']

x_test = pd.concat([x_mean_test, x_cat_test, x_last_test, x_past_target_test['kpu_last']], axis=1)
y_test = test['kpu']

In [114]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(191, 31) (191,) (191, 31) (191,)


In [119]:
CAT_FEATURES

['nationality', 'place_of_birth', 'previous_place_of_living', 'bite_type']

In [117]:
x_train.columns

Index(['not_registered', 'num_caries', 'num_filling_caries',
       'num_filling_no_caries', 'num_healthy', 'num_removed_caused_by_caries',
       'num_removed_other_reasons', 'prosthesis crown veneer',
       'sealed fissure', 'uncut tooth', 'fluorosis', 'sum_hygiene_index',
       'right_age', 'nationality', 'place_of_birth',
       'previous_place_of_living', 'bite_type', 'not_registered last',
       'num_caries last', 'num_filling_caries last',
       'num_filling_no_caries last', 'num_healthy last',
       'num_removed_caused_by_caries last', 'num_removed_other_reasons last',
       'prosthesis crown veneer last', 'sealed fissure last',
       'uncut tooth last', 'fluorosis last', 'sum_hygiene_index last',
       'right_age last', 'kpu_last'],
      dtype='object')

# Training Catboost

In [137]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error


train_labels = [13, 14, 15, 16, 17]
model = CatBoostRegressor(iterations=20,
                          learning_rate=0.5,
                          depth=16)


In [138]:
model.fit(x_train, y_train,  train_labels)
y_pred = model.predict(x_test)
score = np.sqrt( mean_squared_error(y_test, y_pred ))
score

0:	learn: 3.5115902	total: 6.89ms	remaining: 131ms
1:	learn: 2.6790160	total: 10.6ms	remaining: 95.7ms
2:	learn: 2.1290815	total: 15.4ms	remaining: 87.2ms
3:	learn: 1.8510430	total: 18.1ms	remaining: 72.4ms
4:	learn: 1.6039060	total: 21ms	remaining: 63.1ms
5:	learn: 1.5009399	total: 27.1ms	remaining: 63.2ms
6:	learn: 1.4291470	total: 29.1ms	remaining: 54ms
7:	learn: 1.2686071	total: 49ms	remaining: 73.5ms
8:	learn: 1.1115867	total: 51.7ms	remaining: 63.1ms
9:	learn: 1.0337476	total: 53.3ms	remaining: 53.3ms
10:	learn: 0.9191611	total: 90.6ms	remaining: 74.1ms
11:	learn: 0.8098716	total: 165ms	remaining: 110ms
12:	learn: 0.7293212	total: 172ms	remaining: 92.7ms
13:	learn: 0.7110649	total: 174ms	remaining: 74.4ms
14:	learn: 0.7040037	total: 175ms	remaining: 58.2ms

Iteration with suspicious time 1.72 sec ignored in overall statistics.
15:	learn: 0.6410112	total: 175ms	remaining: 58.2ms
16:	learn: 0.5819177	total: 186ms	remaining: 34.9ms

Iteration with suspicious time 1.69 sec ignored in

3.6887282968241744

In [144]:
pd.Series(index=x_train.columns, data=model.feature_importances_).sort_values()

previous_place_of_living              0.000000
uncut tooth last                      0.000000
prosthesis crown veneer last          0.000000
num_removed_caused_by_caries          0.000000
prosthesis crown veneer               0.000000
bite_type                             0.000000
place_of_birth                        0.000000
num_removed_other_reasons             0.001134
num_removed_other_reasons last        0.002620
fluorosis                             0.002814
nationality                           0.006750
sealed fissure last                   0.016794
not_registered                        0.034378
uncut tooth                           0.075709
num_removed_caused_by_caries last     0.078177
num_filling_caries                    0.393874
num_filling_caries last               0.479310
sum_hygiene_index last                0.557437
not_registered last                   0.561509
fluorosis last                        2.027313
sum_hygiene_index                     2.544326
sealed fissur